# Use GNN to predict econ outputs

- Replicating regression benchmarks with GNN.
- BOOST the performance. (Only started with simple GNNs)

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import copy
import scipy.sparse as sp
from scipy.sparse import csr_matrix

# GNN packages
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

In [2]:
# read files
with open("../../data/03_processed/place_graph_X.pickle", 'rb') as f:
    X_place = pickle.load(f) # data frame

with open("../../data/03_processed/place_graph_A.pickle", 'rb') as f:
    A_place = pickle.load(f) # sparse matrix

with open("../../data/03_processed/place_graph_weighted_A.pickle", 'rb') as f:
    A_weighted_place = pickle.load(f) # sparse matrix    
    
with open("../../data/03_processed/place_graph_Y.pickle", 'rb') as f:
    Y_place = pickle.load(f) # data frame
    

In [3]:
X_place

,inc_per_capita,property_value_median,pop_total,households,race_white_ratio,race_black_ratio,age_median,travel_driving_ratio,edu_bachelor_ratio
full_bg_fips,,,,,,,,,
250092011001,46400.0,521300.0,544.0,259.0,1.000000,0.000000,52.8,0.728395,0.239669
250092021011,54513.0,464100.0,721.0,248.0,0.970874,0.000000,47.4,0.737931,0.334669
250092021012,48486.0,461900.0,518.0,202.0,0.967181,0.000000,39.9,0.836538,0.413408
250092021013,43408.0,391000.0,805.0,288.0,0.822360,0.045963,35.4,0.761261,0.250000
250092021021,35731.0,403800.0,1181.0,402.0,0.957663,0.029636,33.8,0.902357,0.204301
...,...,...,...,...,...,...,...,...,...
330170870001,25345.0,218500.0,1479.0,549.0,1.000000,0.000000,33.2,0.926868,0.100338
330170870002,24643.0,158700.0,1612.0,630.0,0.984491,0.000000,38.5,0.869505,0.127907
330170870003,28067.0,169300.0,1657.0,597.0,1.000000,0.000000,35.9,0.896261,0.098936


In [4]:
A_place

<3102x3102 sparse matrix of type '<class 'numpy.float64'>'
	with 5290846 stored elements in Compressed Sparse Row format>

In [5]:
Y_place

,inc_per_capita_annual_growth,pop_total_annual_growth,property_value_median_annual_growth
full_bg_fips,,,
250092011001,0.105,-0.094,0.062
250092021011,0.101,0.001,0.050
250092021012,0.078,0.013,0.028
250092021013,0.005,0.034,0.027
250092021021,0.125,-0.037,0.055
...,...,...,...
330170870001,0.094,-0.031,0.020
330170870002,-0.018,0.037,0.044
330170870003,0.161,-0.094,0.107


## Linear reg benchmark (SM Package)

- Predicting income. R2 = 0.71

In [6]:
import statsmodels.api as sm
from sklearn.preprocessing import normalize

In [18]:
# predict income with raw data
input_vars = ['pop_total', 'property_value_median', 
              'households', 'race_white_ratio', 'race_black_ratio', 'age_median', 
              'travel_driving_ratio', 'edu_bachelor_ratio']

# specify X and y
X = sm.add_constant(X_place[input_vars])
output_var = 'inc_per_capita' 
y = X_place[output_var] # here use X_place

# regression on y and X
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:         inc_per_capita   R-squared:                       0.711
Model:                            OLS   Adj. R-squared:                  0.710
Method:                 Least Squares   F-statistic:                     951.1
Date:                Sun, 07 Nov 2021   Prob (F-statistic):               0.00
Time:                        21:09:40   Log-Likelihood:                -33359.
No. Observations:                3102   AIC:                         6.674e+04
Df Residuals:                    3093   BIC:                         6.679e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -1.536e+

In [19]:
# predict income with normalized X and y. 
input_vars = ['pop_total', 'property_value_median', 
              'households', 'race_white_ratio', 'race_black_ratio', 'age_median', 
              'travel_driving_ratio', 'edu_bachelor_ratio']

# specify X and y
X = normalize(X_place[input_vars].values, axis = 0)
X = sm.add_constant(X)
output_var = 'inc_per_capita'
y = normalize(X_place[output_var].values.reshape(-1, 1), axis = 0) # here use X_place

# regression on y and X
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.711
Model:                            OLS   Adj. R-squared:                  0.710
Method:                 Least Squares   F-statistic:                     951.1
Date:                Sun, 07 Nov 2021   Prob (F-statistic):               0.00
Time:                        21:09:43   Log-Likelihood:                 12452.
No. Observations:                3102   AIC:                        -2.489e+04
Df Residuals:                    3093   BIC:                        -2.483e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0059      0.001     -9.976      0.0

## Replicating the linear benchmark with Pytorch

Successfully replicated the sm linear reg by using pytorch. (R2 and parameters are both consistent)

In [10]:
# input x and y.
input_vars = ['pop_total', 'property_value_median', 
              'households', 'race_white_ratio', 'race_black_ratio', 'age_median', 
              'travel_driving_ratio', 'edu_bachelor_ratio']

# specify X and y
X = normalize(X_place[input_vars].values, axis = 0)

output_var = 'inc_per_capita' 
y = normalize(X_place[output_var].values.reshape(-1, 1), axis = 0).reshape(-1,1)


In [11]:
# change types of X and y.
X = X.astype('float32')
y = y.astype('float32')

# tensor
X_tensor = torch.from_numpy(X)
y_tensor = torch.from_numpy(y)

print(X_tensor.size())
print(y_tensor.size())

torch.Size([3102, 8])
torch.Size([3102, 1])


In [13]:
# set up.
import torch.nn.functional as F
from torch.nn import Linear

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
linear_model = Linear(8, 1, bias = True).to(device)
X_tensor = X_tensor.to(device)
y_tensor = y_tensor.to(device)
# optimizer = torch.optim.Adam(linear_model.parameters(), lr=0.01, weight_decay=5e-4)
optimizer = torch.optim.Adam(linear_model.parameters(), lr=0.01)


In [14]:
# train
linear_model.train()

for epoch in range(5000):
    optimizer.zero_grad()
    out = linear_model(X_tensor)
    loss = F.mse_loss(out, y_tensor)
    if epoch%100 == 0:
        print(loss)
    # Here it is. Compute on the WHOLE training set, then evaluate using the mask.
    loss.backward()
    optimizer.step()    

tensor(0.0251, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.0001, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(8.1443e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(6.2233e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(4.7479e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(3.7271e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(3.0750e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.6783e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.4399e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.2924e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.1955e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.1275e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.0772e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.0390e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.0096e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1.9870e-05, device='cuda:0', grad_fn=<MseLossBackward>)


In [15]:
# check the weights
print(linear_model.weight)
print(linear_model.bias)

Parameter containing:
tensor([[-0.3039,  0.6439,  0.3040,  0.1613, -0.0104,  0.2578,  0.0712,  0.1978]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-0.0059], device='cuda:0', requires_grad=True)


In [16]:
# compute R2
# same as above
from sklearn.metrics import r2_score
y_pred = linear_model(X_tensor).cpu().detach().numpy()
y_true = y_tensor.cpu().numpy()
print('R2 from sklearn:', r2_score(y_true, y_pred)) 

R2 from sklearn: 0.7109805092148405


In [17]:
# compute R2 by hand. 
# same results
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

##
y_pred = linear_model(X_tensor)
y_true = y_tensor
after_mse = mse(y_pred, y_true).cpu().detach().numpy()

##
y_average = y_tensor.mean().repeat(len(y_tensor))
before_mse = mse(y_average, y_true).cpu().detach().numpy()

## R2
print(after_mse)
print(before_mse)
print('R2 from my own formula:', 1 - after_mse/before_mse) # Hmm...


1.9089197e-05
6.604814e-05
R2 from my own formula: 0.7109805345535278


## Use Linear + Conv to improve R2.

- Spatial autocorrelation should be able to help. But coding needs to be right. 
- The adjacency matrix is critical. I don't think a fully connected graph can work well.


In [20]:
# describe the weighted adjacency matrix
A_weighted_array = A_weighted_place.toarray()
print(np.sum(A_weighted_array))
pd.DataFrame(A_weighted_array.reshape(-1, 1)).describe()

5540883983.0


,0
count,9.622404e+06
mean,5.758316e+02
std,2.983038e+04
min,0.000000e+00
25%,0.000000e+00
50%,1.000000e+00
75%,2.600000e+01
max,2.358477e+07


In [21]:
# create a threshold to filter the weighting array.
epsilon = 100.0 # you can change it.

import copy
A_threshold = copy.copy(A_weighted_array)
smaller_than_threshold_mask = A_threshold < epsilon
A_threshold[smaller_than_threshold_mask] = 0.0
larger_than_threshold_mask = A_threshold > epsilon
A_threshold[larger_than_threshold_mask] = 1.0

print("Num of edges in the new adjacency matrix: ", np.sum(A_threshold))
print("Num of edges in the initial adjacency matrix: ", np.sum(A_place))
print("Total number of potential edges: ", A_place.shape[0]*A_place.shape[1])

# Q: what is the right density in a graph?

A_threshold

Num of edges in the new adjacency matrix:  2073356.0
Num of edges in the initial adjacency matrix:  5290846.0
Total number of potential edges:  9622404


array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [23]:
# remove self loop.
print(np.sum(np.diagonal(A_threshold))) # ! Many self-loops do not reach the threshold!
np.fill_diagonal(A_threshold, 0.0)
print(np.sum(np.diagonal(A_threshold)))

0.0
0.0


In [24]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from scipy.sparse import csr_matrix

In [25]:
# input x and y.
input_vars = ['pop_total', 'property_value_median', 
              'households', 'race_white_ratio', 'race_black_ratio', 'age_median', 
              'travel_driving_ratio', 'edu_bachelor_ratio']

# specify X and y
X = normalize(X_place[input_vars].values, axis = 0)

output_var = 'inc_per_capita'
y = normalize(X_place[output_var].values.reshape(-1, 1), axis = 0).reshape(-1,1)

# change types of X and y.
X = X.astype('float32')
y = y.astype('float32')

# X and y tensor
X_tensor = torch.from_numpy(X)
y_tensor = torch.from_numpy(y)

print(X_tensor.size())
print(y_tensor.size())


torch.Size([3102, 8])
torch.Size([3102, 1])


In [26]:
# Use the scipy crs to get the list of (row, col) indicators.
row_idx = csr_matrix(A_threshold).tocoo().row
col_idx = csr_matrix(A_threshold).tocoo().col
data_idx = csr_matrix(A_threshold).tocoo().data

print(len(data_idx))
print(len(col_idx))
print(len(row_idx))

# test self loops
count = 0
for i in range(len(col_idx)):
    if col_idx[i] == row_idx[i]:
        count += 1
print("Number of self-loops is: ", count)


1436128
1436128
1436128
Number of self-loops is:  0


In [27]:
# Create the data object
edge_index = torch.tensor([list(row_idx)+list(col_idx), list(col_idx)+list(row_idx)], dtype = torch.long)
data = Data(edge_index=edge_index, x = X_tensor, y = y_tensor)
data 

Data(x=[3102, 8], edge_index=[2, 2872256], y=[3102, 1])

In [28]:
# check the properties of the data
print(data.is_undirected())
print(data.has_self_loops())
print(data.num_node_features)

True
False
8


In [29]:
# modeling
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, Linear, GATConv

In [30]:
# start with the simplest case.
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(data.num_node_features, 1, bias = True)
        self.lin1 = Linear(-1, 1, bias = True)
#         self.conv2 = GCNConv(16, dataset.num_classes)
#         self.conv2 = GCNConv(16, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
#         x = self.lin1(x)
        x = self.conv1(x, edge_index) + self.lin1(x)
#         x = F.relu(x)
#         x = F.dropout(x, training=self.training)
#         x = self.conv2(x, edge_index)
        return x


In [31]:
class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GATConv((-1, -1), hidden_channels, add_self_loops=False)
        self.lin1 = Linear(-1, hidden_channels)
        self.conv2 = GATConv((-1, -1), out_channels, add_self_loops=False)
        self.lin2 = Linear(-1, out_channels)

    def forward(self, data):
        x = self.conv1(data.x, data.edge_index) + self.lin1(data.x) # Hmm, add linear here! Consider using it for your naive GNN.
        x = x.relu()
        x = self.conv2(x, data.edge_index) + self.lin2(data.x)
        return x

# model = GAT(hidden_channels=64, out_channels=1)


### Naive GCN Model

In [32]:
# set up
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# model_GAT = GAT(hidden_channels=64, out_channels=1).to(device)
model_GCN = GCN().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model_GCN.parameters(), lr=0.01, weight_decay=9e-6)
# optimizer = torch.optim.Adam(model_GAT.parameters(), lr=0.01)


In [33]:
# train
model_GCN.train()
for epoch in range(10000):
    optimizer.zero_grad()
    out = model_GCN(data)
    loss = F.mse_loss(out, data.y)
#     print(loss)
    if epoch % 100 == 0:
        print(loss)
    loss.backward()
    optimizer.step()


tensor(0.0952, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.0002, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.0001, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(6.3758e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(4.3001e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(3.5425e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(3.2259e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(3.0356e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.8873e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.7615e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.6529e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.5585e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.4764e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.4050e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.3430e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.2892e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tens

In [34]:
# compute R2
# Hmm...low R2.
from sklearn.metrics import r2_score
y_pred = model_GCN(data).cpu().detach().numpy()
y_true = y_tensor.cpu().numpy()
print('R2 from sklearn:', r2_score(y_true, y_pred)) 

R2 from sklearn: 0.7060304306113402


### Naive GAT model

In [35]:
# set up
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model_GAT = GAT(hidden_channels=64, out_channels=1).to(device)
# model_GCN = GCN().to(device)
data = data.to(device)
# optimizer = torch.optim.Adam(model_GCN.parameters(), lr=0.01, weight_decay=9e-6)
optimizer = torch.optim.Adam(model_GAT.parameters(), lr=0.01)


In [36]:
# train GAT
model_GAT.train()
for epoch in range(10000):
    optimizer.zero_grad()
    out = model_GAT(data)
    loss = F.mse_loss(out, data.y)
#     print(loss)
    if epoch % 100 == 0:
        print(loss)
    loss.backward()
    optimizer.step()


tensor(0.0047, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.0004, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.0002, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(8.1024e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(4.5971e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(3.3750e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.8449e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.5478e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.3547e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.2226e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.1304e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.0655e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.0198e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1.9874e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1.9642e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1.9473e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tens

KeyboardInterrupt: 

In [37]:
# compute R2
# slightly better
from sklearn.metrics import r2_score
y_pred = model_GAT(data).cpu().detach().numpy()
y_true = y_tensor.cpu().numpy()
print('R2 from sklearn:', r2_score(y_true, y_pred)) 

R2 from sklearn: 0.7144131769427193
